In [0]:
%pip install python-dotenv

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import DataFrame
from pyspark.sql.functions import min as spark_min, max as spark_max
from pyspark.sql.types import ArrayType, DoubleType, StringType, StructType, StructField, IntegerType, LongType, FloatType

from delta.tables import DeltaTable

from sedona.spark import *

import random
import geopandas as gpd
from pathlib import Path
import time
import os
import numpy as np
from PIL import Image, ImageDraw
from dotenv import load_dotenv
import requests

In [0]:
# config
catalog_dev = "`land_topografisk-gdb_dev`"
schema_dev = "ai2025"

spark.sql(f"USE CATALOG {catalog_dev}")
spark.sql(f"USE SCHEMA {schema_dev}")

log_table = "logs_processed_gdbs"
bronze_table = "polygons_bronze"
silver_table = "polygons_silver"
buffer = 20  # buffer rundt polygonet gir unøyaktig størrelse hvis f.eks. pikselverdi er tenk å være 0.25kvm

In [0]:
df_bronze = spark.read.table(bronze_table)
df_bronze.display(5)

In [0]:
def read_table_to_wkt() -> DataFrame:
    """
    Read a Spark DataFrame column containing WKT and return a Spark DataFrame with the
    """
    df_bronze = spark.read.table(bronze_table).withColumn("geometry", F.expr("ST_GeomFromWKT(geometry)"))
    return df_bronze

In [0]:
def make_envelope(df: DataFrame) -> DataFrame:
    """
    Make a envelope based on the geom.
    """
    return df.withColumn("envelope", F.expr("ST_Boundary(geometry)"))

In [0]:
def random_adjusted_bbox_centered(
    envelope: list,
    min_size: int = 256,
    max_size: int = 256,
    margin: int = 30,
    max_offset: float = 80,  # margin - få meter
    max_attempts: int = 10
) -> list:
    """
    Lager en justert BBOX som garanterer at polygonet (envelope) er synlig og ligger innenfor BBOX,
    med en viss variasjon i størrelse og plassering.

    Args:
        envelope: [xmin, ymin, xmax, ymax]
        min_size: Minimum størrelse på BBOX (f.eks. 256 m)
        max_size: Maksimal størrelse (f.eks. 512 m)
        margin: Buffer rundt polygonet (f.eks. 40 m)
        max_offset: Maks tilfeldig forskyvning fra sentrum (f.eks. ±40 m)
        max_attempts: Antall forsøk før feilmelding

    Returns:
        (bbox: list, bbox_str: str) – bbox = [xmin, ymin, xmax, ymax]
    """
    import random

    xmin, ymin, xmax, ymax = envelope
    poly_width = xmax - xmin
    poly_height = ymax - ymin

    # Beregn ønsket BBOX-størrelse
    bbox_size = max(poly_width, poly_height) + margin * 2
    bbox_size = min(max(bbox_size, min_size), max_size)
    half_size = bbox_size / 2

    # Polygonets sentrum
    center_x_orig = (xmin + xmax) / 2
    center_y_orig = (ymin + ymax) / 2

    for _ in range(max_attempts):
        dx = random.uniform(-max_offset, max_offset)
        dy = random.uniform(-max_offset, max_offset)

        center_x = center_x_orig + dx
        center_y = center_y_orig + dy

        # Lag BBOX
        adjusted_xmin = center_x - half_size
        adjusted_xmax = center_x + half_size
        adjusted_ymin = center_y - half_size
        adjusted_ymax = center_y + half_size

        # Sjekk at hele polygonet er innenfor den justerte BBOX-en
        if (adjusted_xmin <= xmin and adjusted_ymin <= ymin and
            adjusted_xmax >= xmax and adjusted_ymax >= ymax):
            bbox = [adjusted_xmin, adjusted_ymin, adjusted_xmax, adjusted_ymax]
            bbox_str = "_".join(f"{v:.6f}" for v in bbox)
            return bbox, bbox_str

    raise ValueError("Fant ikke gyldig adjusted_bbox etter flere forsøk")


In [0]:
def make_bbox(df: DataFrame, column_name: str) -> DataFrame: 
    """
    Make a bounding box from a Spark DataFrame based on the envelope.
    """
    df = df.withColumn(
    "bbox",
    F.expr(f"""
    array(
        ST_X(ST_Centroid(envelope)) - (GREATEST(ST_XMax(envelope) - ST_XMin(envelope), ST_YMax(envelope) - ST_YMin(envelope)) / 2 + {buffer}),
        ST_Y(ST_Centroid(envelope)) - (GREATEST(ST_XMax(envelope) - ST_XMin(envelope), ST_YMax(envelope) - ST_YMin(envelope)) / 2 + {buffer}),
        ST_X(ST_Centroid(envelope)) + (GREATEST(ST_XMax(envelope) - ST_XMin(envelope), ST_YMax(envelope) - ST_YMin(envelope)) / 2 + {buffer}),
        ST_Y(ST_Centroid(envelope)) + (GREATEST(ST_XMax(envelope) - ST_XMin(envelope), ST_YMax(envelope) - ST_YMin(envelope)) / 2 + {buffer})
    )
    """) # Lager BBOX kolonne som kan brukes videre i WMSene på en enkel måte.
    )
    df = df.withColumn(
        "Polygons",
        F.expr("ST_MakeEnvelope(bbox[0], bbox[1], bbox[2], bbox[3])")
    ) # Lager polygons basert på bboxen 
    df = df.withColumn(
        "adjusted_struct",
        adjusted_bbox_udf(F.col("bbox"))
    ).withColumn(
        "Adjusted_bbox", F.col("adjusted_struct.bbox")
    ).withColumn(
        "bbox_str", F.col("adjusted_struct.bbox_str")
    ).drop("envelope")

    return df

In [0]:
def generate_dom_url(bbox_str):
    width, height = [512, 512]
    return (
        f"https://wms.geonorge.no/skwms1/wms.hoyde-dom-nhm-25833?request=GetMap&Format=image/png&"
        f"GetFeatureInfo=text/plain&CRS=EPSG:25833&Layers=NHM_DOM_25833:skyggerelieff&"
        f"BBOX={bbox_str}&width={width}&height={height}"
    )

def generate_image_url(bbox_str):
    width, height = [512, 512]
    return (
        f"https://wms.geonorge.no/skwms1/wms.nib?VERSION=1.3.0"
        f"&service=WMS&request=GetMap&Format=image/png&"
        f"GetFeatureInfo=text/plain&CRS=EPSG:25833&Layers=ortofoto&"
        f"BBox={bbox_str}&width={width}&height={height}&TICKET="
    )

def dom_file_exists(id: str) -> str:
    path = f"/Volumes/land_topografisk-gdb_dev/external_dev/static_data/DL_SNUPLASSER/doms/dom_{id}.png"
    return "DOWNLOADED" if os.path.exists(path) else "PENDING"

def image_file_exists(id: str) -> str:
    path = f"/Volumes/land_topografisk-gdb_dev/external_dev/static_data/DL_SNUPLASSER/images/image_{id}.png"
    return "DOWNLOADED" if os.path.exists(path) else "PENDING"

def mask_file_exists(id: str) -> str:
    path = f"/Volumes/land_topografisk-gdb_dev/external_dev/static_data/DL_SNUPLASSER/labels/mask_{id}.png"
    return "GENERATED" if os.path.exists(path) else "PENDING"

In [0]:
def write_delta_table(sdf: DataFrame):
    """
    Write Spark DataFrame to Delta table.
    Automatically updates or inserts all columns.
    """
    if not spark.catalog.tableExists(silver_table):
        sdf.write.format("delta") \
            .option("mergeSchema", "true") \
            .mode("overwrite") \
            .saveAsTable(silver_table)
    else:
        delta_tbl = DeltaTable.forName(spark, silver_table)
        delta_tbl.alias("target") \
            .merge(
                source=sdf.alias("source"),
                condition="target.row_hash = source.row_hash"
            ) \
            .whenMatchedUpdateAll() \
            .whenNotMatchedInsertAll() \
            .execute()

In [0]:
def plot(df: DataFrame, column_name: str, plot_column: str):
    """
    Plot the bbox.
    """
    bbox_gdf = gpd.GeoDataFrame(
    df.toPandas(),
    geometry=column_name,
    crs="EPSG:25833",  # Bruker ETRS89 / UTM sone 33N (Norge)
    )
    return bbox_gdf

In [0]:
adjusted_bbox_schema = StructType([
    StructField("bbox", ArrayType(DoubleType())),
    StructField("bbox_str", StringType())
])
adjusted_bbox_udf = F.udf(lambda envelope: random_adjusted_bbox_centered(envelope), adjusted_bbox_schema)
generate_dom_url_udf = F.udf(generate_dom_url, StringType())
generate_image_url_udf = F.udf(generate_image_url, StringType())
dom_file_status_udf = F.udf(dom_file_exists, StringType())
image_file_status_udf = F.udf(image_file_exists, StringType())
mask_file_exists_udf = F.udf(mask_file_exists, StringType())

BASE_PATH = "/Volumes/land_topografisk-gdb_dev/external_dev/static_data/DL_SNUPLASSER"

df = read_table_to_wkt()
df = make_envelope(df)
df = make_bbox(df, "envelope")

for dt in ["image", "dom", "mask"]:
    df = df.withColumn(
        f"{dt}_path",
        F.concat(
            F.lit(f"{BASE_PATH}/{dt}_"),
            F.col("row_hash"),
            F.lit(".png")
        )
    )
df = df.withColumn("image_wms", generate_image_url_udf("Adjusted_bbox")) \
       .withColumn("dom_wms", generate_dom_url_udf("Adjusted_bbox")) \
       .withColumn("image_status", image_file_status_udf("row_hash")) \
       .withColumn("dom_status", dom_file_status_udf("row_hash")) \
       .withColumn("mask_status", mask_file_exists_udf("row_hash")) \
       .withColumn("lastet_tid", F.current_timestamp())

gdf = plot(df, "Polygons", "Vegtyper")
write_delta_table(df)


In [0]:
df.display()

In [0]:
silverdf = spark.read.table(silver_table)
silverdf.display()

In [0]:
df_retable = spark.read.table(silver_table)
df_overview = df_retable.drop(
    "Shape_Length",
    "Shape_Area",
    "Vegtyper",
    "Snuplasstype",
    "Parkering",
    "geometry",
    "ingest_time",
    "source_file",
    "source_layer",
    "bbox",
    "Polygons",
    "adjusted_struct",
    "Adjusted_bbox",
    "bbox_str",
    "image_wms",
    "dom_wms",
    "lastet_tid"
)

df_overview.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable("polygons_status_overview")

In [0]:
# Uncomment under hvis man ønsker å se på plotten
# gdf.explore(column="Vegtyper", tooltip="Vegtyper", popup=True, cmap="Set1")

In [0]:
#spark.sql(f"DROP TABLE IF EXISTS {silver_table}")